<center><font size=5>多因子对比示例</font></center>

仅仅比单因子多一个循环

加载各种包

In [1]:
import matplotlib.pyplot as plt
import sys
sys.path.append('..')  # 文件位于

import factors  # 因子计算
import data_utils  # 数据读取
import performance as perf  # IC计算
import plotting  #绘图

# 1. 初始化DataLoader 

DataLoader实现了对csv文件的批量读取，会按照\[年\]\[月\]保存在dl中。
一些重要函数：
- init_month_df: 加载目标文件夹下的所有zip文件。这些文件读取后会被保存在dataloader中。注意，

In [2]:
dl = data_utils.DataLoader()
dl.init_month_df('../../data/15m_data/')
# 计算未来收益率
data_utils.compute_future_rtn_for_all(dl)

In [3]:
import pandas as pd
import numpy as np
from pandas_ta.core import macd, rsi, atr, vwap, ohlc4, bop

In [4]:
def cal_Z_score(factor_df, column_lists, span):
    '''Z-score'''

    for column_name in column_lists:
        mean_20 = factor_df[column_name].ewm(span, adjust=False).mean()
        std_20 = factor_df[column_name].ewm(96, adjust=False).std()  # todo 超参数搜索
        factor_df[f'{column_name}_Z'] = (factor_df[column_name].values - mean_20) / std_20

    return factor_df

In [5]:
from pandas_ta.core import adx, cci, macd, rsi, obv, vwap, atr, bop, ohlc4

In [7]:
@factors.handle_dataloader_data
def add_new_factor(raw_df:pd.DataFrame)-> pd.DataFrame:
    '''
    加入新因子的模板。
    :param raw_df: 原始数据
    :param factor_name: 因子名称
    :return: 原始数据+因子
    '''
    data_df = raw_df[['open', 'close', 'high', 'low', 'volume','taker_buy_volume']]
    data_df.index = pd.to_datetime(raw_df.open_time, unit='ms')
    temp_factor_df = pd.DataFrame()  # 暂存特征，避免index不一致导致的错误
    temp_factor_df['trend_adx'] = adx(data_df['high'], data_df['low'], data_df['close'])['ADX_14']
    temp_factor_df['trend_cci'] = cci(data_df['high'], data_df['low'], data_df['close'])
    temp_factor_df['macd'] = macd(data_df['close'])['MACD_12_26_9']
    temp_factor_df['momentum_rsi'] = rsi(data_df['close'])
    temp_factor_df['volume_obv'] = obv(data_df['close'], data_df['volume'])
    temp_factor_df['volume_vwap'] = vwap(data_df['high'], data_df['low'], data_df['close'], data_df['volume'])
    temp_factor_df['volatility_atr'] = atr(data_df['high'], data_df['low'], data_df['close'])
    temp_factor_df['bop'] = bop(data_df['open'], data_df['high'], data_df['low'],
                            data_df['close'])  # (open - close)/(high - low)
    temp_factor_df['ohlc4'] = ohlc4(data_df['open'], data_df['high'], data_df['low'], data_df['close'])
    temp_factor_df['buy_taker_prop'] = data_df['taker_buy_volume']/data_df['volume']
    temp_factor_df.reset_index(drop=True, inplace=True)
    raw_df = pd.concat([raw_df, temp_factor_df], axis=1)
    
    
    cal_Z_score(raw_df,['buy_taker_prop'], 100)
#     raw_df['bw_rtn_10'] = raw_df['fwd_rtn_10'].shift(10)

    return raw_df

In [ ]:
z_targets = ['close', 'trend_adx', 'trend_cci', 'macd', 'momentum_rsi', \
                        'volume_obv', 'volume_vwap', 'volatility_atr', \
                        'bop', 'ohlc4', 'volume']

In [ ]:
Z_cols = [f'{col}_Z' for col in z_targets]

In [8]:
# 计算因子
add_new_factor(dl)

In [ ]:
for colname in z_targets:
    for col in [colname +'_Z', colname]:
        print(f'feat name: {col}')
        tests_factor_name = col
        # 计算因子IC
        month_mean_ic = perf.cal_monthly_mean_ic(dl, factor_name=tests_factor_name, use_real_rtn=True)
        # 保存记录
        dl.set_record_df(kind='Info', factor_name=tests_factor_name, result_df=month_mean_ic)
        # 读取记录，并且展示作图
        month_mean_ic = dl.get_record_df(kind='Info', factor_name=tests_factor_name)
        plotting.plot_information_table(month_mean_ic)
